In [12]:
# Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf 
import plotly.express as px
from sklearn import metrics
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model

from forecasting_func import timeSeriesMultivariate, timeSeriesEvaluationMetrics

In [13]:
# Reading CSV dataset with ANSI encoding as it includes German Tokens

df = pd.read_csv(r"dataset/preprocessed_accidents.csv", encoding = "ANSI")
df = df.iloc[:, 1:]
df

,Category,AccidentType,Year,Month,Value
0,Traffic Accidents,injured and killed,2000,200012,515
1,Traffic Accidents,injured and killed,2000,200011,578
2,Traffic Accidents,injured and killed,2000,200010,615
3,Traffic Accidents,injured and killed,2000,200009,675
4,Traffic Accidents,injured and killed,2000,200008,647
...,...,...,...,...,...
1759,Alcohol Accidents,subtotal,2020,202005,40
1760,Alcohol Accidents,subtotal,2020,202004,26
1761,Alcohol Accidents,subtotal,2020,202003,27
1762,Alcohol Accidents,subtotal,2020,202002,40


In [14]:
len(pd.unique(df['Month']))

252

This means there are 252 uniques dates in this dataset. So we have to iterate every 252 records and mark their respective category and types of deaths

In [15]:
date_vals = {}
for index, row in df.iterrows():
    date_vals[row['Month']] = []
for index, row in df.iterrows():
    date_vals[row['Month']].append(row['Value'])

In [16]:
for date, vals in date_vals.items():
    date_vals[date][2] = vals[2] - vals[1] - vals[0]
    date_vals[date][4] = vals[4] - vals[3]
    date_vals[date][6] = vals[6] - vals[5]
    
date_vals

{200012: [515, 403, 2871, 45, 905, 31, 54],
 200011: [578, 461, 2892, 56, 961, 24, 47],
 200010: [615, 493, 3009, 47, 1031, 31, 52],
 200009: [675, 557, 2575, 68, 910, 45, 39],
 200008: [647, 525, 2215, 74, 807, 32, 45],
 200007: [744, 610, 2928, 67, 1035, 37, 62],
 200006: [685, 555, 2089, 58, 784, 32, 25],
 200005: [675, 568, 2880, 69, 1017, 44, 52],
 200004: [557, 441, 2319, 57, 777, 26, 52],
 200003: [509, 414, 2860, 48, 926, 18, 55],
 200002: [419, 329, 2433, 35, 855, 19, 34],
 200001: [434, 321, 2693, 34, 801, 37, 41],
 200112: [507, 377, 3105, 39, 902, 23, 43],
 200111: [568, 436, 3018, 32, 969, 25, 43],
 200110: [717, 581, 2952, 85, 1019, 37, 40],
 200109: [643, 497, 2799, 40, 913, 48, 59],
 200108: [600, 510, 2241, 57, 757, 21, 39],
 200107: [921, 755, 2612, 83, 964, 35, 36],
 200106: [679, 560, 2476, 78, 827, 46, 22],
 200105: [817, 655, 2656, 67, 1014, 32, 53],
 200104: [578, 454, 2533, 68, 829, 32, 36],
 200103: [535, 426, 3009, 43, 919, 36, 27],
 200102: [439, 343, 2366, 5

In [17]:
df = pd.DataFrame.from_dict(date_vals, orient = 'index')
df

,0,1,2,3,4,5,6
200012,515,403,2871,45,905,31,54
200011,578,461,2892,56,961,24,47
200010,615,493,3009,47,1031,31,52
200009,675,557,2575,68,910,45,39
200008,647,525,2215,74,807,32,45
...,...,...,...,...,...,...,...
202005,464,416,2313,34,825,26,14
202004,363,314,1574,34,573,15,11
202003,334,270,2221,33,711,13,14
202002,355,306,2610,34,836,19,21


In [18]:
df.columns = [
    'traffic_injured_and_killed', 
    'traffic_with_people',
    'traffic_other',
    'escape_injured_and_killed', 
    'escape_other',
    'alcohol_injured_and_killed',
    'alcohol_other'
]

# df.rename(columns = {
#     '0':'traffic_injured_and_killed', 
#     '1':'traffic_with_people',
#     '2':'traffic_other',
#     '3':'escape_injured_and_killed', 
#     '4':'escape_other',
#     '5':'alcohol_injured_and_killed',
#     '6':'alcohol_other'
# }, inplace = True)

In [19]:
df

,traffic_injured_and_killed,traffic_with_people,traffic_other,escape_injured_and_killed,escape_other,alcohol_injured_and_killed,alcohol_other
200012,515,403,2871,45,905,31,54
200011,578,461,2892,56,961,24,47
200010,615,493,3009,47,1031,31,52
200009,675,557,2575,68,910,45,39
200008,647,525,2215,74,807,32,45
...,...,...,...,...,...,...,...
202005,464,416,2313,34,825,26,14
202004,363,314,1574,34,573,15,11
202003,334,270,2221,33,711,13,14
202002,355,306,2610,34,836,19,21


In [20]:
df.to_csv("./dataset/accident_by_date.csv")

In [22]:
sum(df['traffic_other'])

655482